In [7]:
import arcpy
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from arcpy import env

In [8]:
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\DZ.gdb'

In [9]:
def merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds):
    try:
        # 复制输入要素类
        arcpy.CopyFeatures_management(input_fc, output_fc)
        print(f"已创建输出要素类: {output_fc}")

        # 确保存在面积字段
        if "Area" not in [f.name for f in arcpy.ListFields(output_fc)]:
            arcpy.AddField_management(output_fc, "Area", "DOUBLE")
        
        # 计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print("已计算面积")

        # 创建空间索引
        arcpy.AddSpatialIndex_management(output_fc)

        # 获取小面积图斑
        small_parcels = []
        fields = ["OID@", land_type_field, dz_field, "Area", "SHAPE@"]
        with arcpy.da.SearchCursor(output_fc, fields) as cursor:
            for row in cursor:
                oid, land_type, dz, area, shape = row
                threshold = thresholds.get(dz, min(thresholds.values()))
                if area < threshold:
                    small_parcels.append((oid, land_type, dz, area, shape))

        print(f"找到 {len(small_parcels)} 个小于指定阈值的图斑")

        # 创建numpy数组来存储图斑信息
        dtype = np.dtype([('oid', np.int32), ('land_type', 'U50'), ('dz', 'U10'), ('area', np.float64)])
        parcels_array = np.array([(p[0], p[1], p[2], p[3]) for p in small_parcels], dtype=dtype)

        # 对图斑按地类和面积排序
        sorted_indices = np.argsort(parcels_array, order=['land_type', 'area'])
        sorted_parcels = [small_parcels[i] for i in sorted_indices]

        # 分批处理
        batch_size = 1000
        total_batches = (len(sorted_parcels) + batch_size - 1) // batch_size

        def process_batch(batch):
            merged_count = 0
            with arcpy.da.Editor(arcpy.Describe(output_fc).path) as editor:
                for small_parcel in batch:
                    oid, land_type, dz, area, shape = small_parcel
                    
                    # 选择相邻且相同地类的图斑
                    where_clause = f"{land_type_field} = '{land_type}' AND OBJECTID <> {oid}"
                    arcpy.SelectLayerByAttribute_management("output_layer", "NEW_SELECTION", where_clause)
                    arcpy.SelectLayerByLocation_management("output_layer", "BOUNDARY_TOUCHES", shape, selection_type="SUBSET_SELECTION")
                    
                    # 查找最大的相邻图斑
                    max_area = 0
                    max_oid = None
                    with arcpy.da.SearchCursor("output_layer", ["OID@", "Area"]) as neighbor_cursor:
                        for neighbor in neighbor_cursor:
                            if neighbor[1] > max_area:
                                max_area = neighbor[1]
                                max_oid = neighbor[0]
                    
                    if max_oid:
                        try:
                            # 合并几何
                            where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                            with arcpy.da.UpdateCursor(output_fc, ["OID@", "SHAPE@", land_type_field], where_clause) as update_cursor:
                                shapes = []
                                for update_row in update_cursor:
                                    shapes.append(update_row[1])
                                    if update_row[0] == oid:
                                        update_cursor.deleteRow()
                                
                                if len(shapes) == 2:
                                    merged_shape = shapes[0].union(shapes[1])
                                    update_cursor.reset()
                                    for update_row in update_cursor:
                                        if update_row[0] == max_oid:
                                            update_cursor.updateRow([max_oid, merged_shape, land_type])
                                            merged_count += 1
                                            break
                        except Exception as e:
                            print(f"处理 OID {oid} 时出错: {str(e)}")
                
                # 清除选择
                arcpy.SelectLayerByAttribute_management("output_layer", "CLEAR_SELECTION")
            
            return merged_count

        # 创建图层
        arcpy.MakeFeatureLayer_management(output_fc, "output_layer")

        # 使用线程池并行处理批次
        total_merged = 0
        with ThreadPoolExecutor(max_workers=4) as executor:
            future_to_batch = {executor.submit(process_batch, sorted_parcels[i:i+batch_size]): i//batch_size for i in range(0, len(sorted_parcels), batch_size)}
            for future in as_completed(future_to_batch):
                batch_index = future_to_batch[future]
                try:
                    merged_count = future.result()
                    total_merged += merged_count
                    print(f"批次 {batch_index + 1}/{total_batches} 完成，合并了 {merged_count} 个图斑")
                except Exception as exc:
                    print(f"批次 {batch_index + 1} 生成了一个异常: {exc}")

        # 重新计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print(f"操作完成。总共合并了 {total_merged} 个小面积图斑到相邻的相同地类图斑中。")
        print(f"结果保存在要素类: {output_fc}")

    except Exception as e:
        print(f"发生错误: {str(e)}")
        print(traceback.format_exc())



In [10]:
# 使用示例
if __name__ == "__main__":
    input_fc = r"D:\ArcGISProjects\workspace\shbyq\DZ.gdb\DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_COPY"
    output_fc = r"D:\ArcGISProjects\workspace\shbyq\DZ.gdb\DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_COPY_END"
    land_type_field = "DLMC"
    dz_field = "DZ"
    thresholds = {
        "01": 50,
        "03": 100,
        "04": 100,
        # 可以继续添加其他DZ值的阈值
    }

    merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds)

已创建输出要素类: D:\ArcGISProjects\workspace\shbyq\DZ.gdb\DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_COPY_END
已计算面积
